In [29]:
import json
from itertools import product
import numpy as np
# Data structure to organize pair conditions
cam_velocities_per_motion_condition_given_obj_vel = {
    "contrast": {
        0: 0,
        7.5: 0,
        15.0: 0,
        30.0: 0,
    }
}

object_velocities = [
    7.5,
    15.0, 
    30.0
    ]

object_directions = {
    "horizontal_left": (-1, 0),
    "horizontal_right": (1, 0)
}

initial_ball_placements = {
    "horizontal_left": {
        0: (.5, 0.5),
        7.5: (.5, 0.5),
        15.0: (.5, 0.5),
        30.0: (.5, 0.5),
    },
    "horizontal_right": {
        0: (.5, 0.5),
        7.5: (.5, 0.5),
        15.0: (.5, 0.5),
        30.0: (.5, 0.5),
    }
}

initial_crosshair_placements = {
    "horizontal_left": {
        0: (.5, 0.5),
        7.5: (.5, 0.5),
        15.0: (.5, 0.5),
        30.0: (.5, 0.5),
    },
    "horizontal_right": {
        0: (.5, 0.5),
        7.5: (.5, 0.5),
        15.0: (.5, 0.5),
        30.0: (.5, 0.5),
    }
}

duration_mapping = {
    "horizontal_left": {
        0: (1.0, 1.5),
        7.5: (1.0, 1.5),
        15.0: (1.0, 1.5),
        30.0: (1.0, 1.5),
    },
    "horizontal_right": {
        0: (1.0, 1.5),
        7.5: (1.0, 1.5),
        15.0: (1.0, 1.5),
        30.0: (1.0, 1.5),
    }
}

contrast_JND = 0.0166

contrast_thresholds = [
    1 + contrast_JND, 
    1 + contrast_JND * 2, 
    1 + contrast_JND * 4, 
    1 + contrast_JND * 8,
    # 1.5
    ]

# Helper function to generate trial segments
def generate_segments_2_speeds(object_direction, object_velocity, duration, contrast_threshold):    
    return [
        {
            "object_motion": object_direction,
            "object_speed": object_velocity,
            "camera_motion": (0, 0),
            "camera_speed": 0,
            "duration": duration, 
            "contrast_threshold_multiplier": contrast_threshold
        }
    ]

# Generate trial data
unique_trials = []
trial_id = 0
for obj_velocity, (obj_motion_type, obj_direction) in product(
        object_velocities,
        object_directions.items()):
    
        for contrast_threshold in contrast_thresholds:
            initial_ball_placement = initial_ball_placements[obj_motion_type][obj_velocity]
            initial_crosshair_placement = initial_crosshair_placements[obj_motion_type][obj_velocity]
            duration = duration_mapping[obj_motion_type][obj_velocity]
        
            segments_1 = generate_segments_2_speeds(obj_direction, obj_velocity, duration, contrast_threshold)

            trial_positive = {
                "trial_id": trial_id,
                "initial_ball_placement": {"x": initial_ball_placement[0], "y": initial_ball_placement[1]},
                "initial_crosshair_placement": {"x": initial_crosshair_placement[0], "y": initial_crosshair_placement[1]},
                "segments": segments_1
            }

            print(trial_positive)
 
            unique_trials.append(trial_positive)
            trial_id += 1

                
# Write trials to JSON
output_file = "trial_ids_segments/trials_FINAL_monde.json"
with open(output_file, mode='w') as file:
    json.dump(unique_trials, file, indent=4)

print(f"Trials saved to {output_file}")
print(len(unique_trials))


{'trial_id': 0, 'initial_ball_placement': {'x': 0.5, 'y': 0.5}, 'initial_crosshair_placement': {'x': 0.5, 'y': 0.5}, 'segments': [{'object_motion': (-1, 0), 'object_speed': 7.5, 'camera_motion': (0, 0), 'camera_speed': 0, 'duration': (1.0, 1.5), 'contrast_threshold_multiplier': 1.0166}]}
{'trial_id': 1, 'initial_ball_placement': {'x': 0.5, 'y': 0.5}, 'initial_crosshair_placement': {'x': 0.5, 'y': 0.5}, 'segments': [{'object_motion': (-1, 0), 'object_speed': 7.5, 'camera_motion': (0, 0), 'camera_speed': 0, 'duration': (1.0, 1.5), 'contrast_threshold_multiplier': 1.0332}]}
{'trial_id': 2, 'initial_ball_placement': {'x': 0.5, 'y': 0.5}, 'initial_crosshair_placement': {'x': 0.5, 'y': 0.5}, 'segments': [{'object_motion': (-1, 0), 'object_speed': 7.5, 'camera_motion': (0, 0), 'camera_speed': 0, 'duration': (1.0, 1.5), 'contrast_threshold_multiplier': 1.0664}]}
{'trial_id': 3, 'initial_ball_placement': {'x': 0.5, 'y': 0.5}, 'initial_crosshair_placement': {'x': 0.5, 'y': 0.5}, 'segments': [{'o

No step ramp, single segment

In [16]:
import json
import pandas as pd

# Load the CSV and JSON files
csv_file = "GazeDataContrast/gaze_data_contrast_velocity_monde.csv"  # Replace with your CSV file name
incorrect_json_file = "trials_segments/generated_trials_change_velocity_pilot.json"  # Replace with your JSON file name
correct_json_file = "trial_ids_segments/trials_change_velocity_lol.json"  # Replace with your JSON file name

# Load the CSV data into a DataFrame
df = pd.read_csv(csv_file)

# Load the incorrect and correct JSON data
with open(incorrect_json_file, 'r') as f:
    incorrect_json = json.load(f)

with open(correct_json_file, 'r') as f:
    correct_json = json.load(f)

# Function to compare each segment
def compare_segments(segments1, segments2):
    if len(segments1) != len(segments2):
        return False
    for seg1, seg2 in zip(segments1, segments2):
        if seg1 != seg2:
            return False
    return True

# Function to find the correct trial ID by comparing JSON fields
def find_correct_trial_id(incorrect_trial):
    for correct_trial in correct_json:
        if (incorrect_trial["initial_ball_placement"] == correct_trial["initial_ball_placement"] and
            incorrect_trial["initial_crosshair_placement"] == correct_trial["initial_crosshair_placement"] and
            compare_segments(incorrect_trial["segments"], correct_trial["segments"])):
            return correct_trial["trial_id"]
    return None  # Return None if no match is found

# Traverse the incorrect trials in order
for trial_index, incorrect_trial in enumerate(incorrect_json):
    correct_id = find_correct_trial_id(incorrect_trial)
    if correct_id is not None:
        # Map to the index in the CSV file where TrialNumber matches trial_index
        df.loc[df['TrialNumber'] == trial_index, 'TrialID'] = correct_id

# Save the updated DataFrame back to a CSV file
updated_csv_file = "monde_updated.csv"
df.to_csv(updated_csv_file, index=False)

print(f"Updated trial IDs saved to {updated_csv_file}")


Updated trial IDs saved to monde_updated.csv


In [24]:
import pandas as pd
import json

# Load the CSV data
csv_file = 'GazeDataColor/gaze_data_color_contrast_niall.csv'
data = pd.read_csv(csv_file)

# Load the JSON file
json_file = 'trials_segments/generated_trials_color_pilot.json'
with open(json_file, 'r') as f:
    trial_details = json.load(f)

# Update the TrialID column in the CSV
def update_trial_ids(data, trial_details):
    # Create a mapping of TrialNumber to TrialID by enumerating the JSON data
    trial_map = {index: trial['trial_id'] for index, trial in enumerate(trial_details)}

    # Ensure TrialNumber column exists
    if 'TrialNumber' not in data.columns:
        raise KeyError("The CSV file does not contain a 'TrialNumber' column.")

    # Ensure TrialID column exists or add it
    if 'TrialID' not in data.columns:
        data['TrialID'] = None

    # Replace TrialID in the CSV based on the mapping
    data['TrialID'] = data['TrialNumber'].map(trial_map)

    return data

# Apply the update
try:
    data = update_trial_ids(data, trial_details)
    # Save the updated CSV
    data.to_csv('updated_gaze_data_color_niall.csv', index=False)
    print("TrialID column successfully updated based on the JSON file.")
except KeyError as e:
    print(f"Error: {e}")
except Exception as e:
    print(f"An unexpected error occurred: {e}")


TrialID column successfully updated based on the JSON file.


In [30]:
import json
import random

# Function to generate a random sequence of trials

def generate_random_trials(input_file, output_file, repetitions):
    # Load trials from input JSON
    with open(input_file, mode='r') as file:
        trials = json.load(file)

    # Create repeated and shuffled trials
    all_trials = []
    for _ in range(repetitions):
        random.shuffle(trials)
        all_trials.extend(trials)

    # # Assign new trial IDs to preserve uniqueness
    # for idx, trial in enumerate(all_trials):
    #     trial["trial_num"] = idx
    # Save randomized trials to output JSON
    with open(output_file, mode='w') as file:
        json.dump(all_trials, file, indent=4)

    print(f"Randomized trials with {repetitions} repetitions saved to {output_file}")
    print(f"Total trials: {len(all_trials)}")

# Parameters

for i in range(0,2):
    input_file = "trial_ids_segments/trials_FINAL_monde.json"  # Path to the input file
    output_file = f"trials_segments/generated_trials_FINAL_monde_{i}.json"  # Path to save the randomized output file
    repetitions = 5  # Number of repetitions

    # Generate the randomized sequence
    generate_random_trials(input_file, output_file, repetitions)


Randomized trials with 5 repetitions saved to trials_segments/generated_trials_FINAL_monde_0.json
Total trials: 120
Randomized trials with 5 repetitions saved to trials_segments/generated_trials_FINAL_monde_1.json
Total trials: 120


Adding Noise

Adding color contrast